# **SentimentFusion: Multimodal Sentiment Analysis**

**Importing necessary libraries**

In [1]:
!pip install transformers

import numpy as np
import pandas as pd
from google.colab import drive
import torchvision.models as models
from PIL import Image
#import transformers
import torch.nn as nn
import os
from torchvision.transforms import transforms
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
import torch
import torchvision
from torchvision import transforms
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score

In [2]:
drive.mount('/content/drive', force_remount=True)
!cd /content/drive/MyDrive/AIProject/

Mounted at /content/drive


In [3]:
data=pd.read_csv("/content/drive/MyDrive/AIProject/sentiment.csv")

In [4]:

data.head(1)

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.


In [5]:
data=data[data.successful==1]
train_data= data[data.split=='train']
test_data= data[data.split=='test']

image_folder = '/content/drive/MyDrive/AIProject/sentiment_images/'
train_image_paths = [image_folder + img_name for img_name in train_data.filename]
test_image_paths = [image_folder + img_name for img_name in test_data.filename]

train_captions= list(train_data.raw)
test_captions= list(test_data.raw)

**Feature Extraction using Resent-50 and BERT:**

In [6]:
# Define ResNet model
resnet = torchvision.models.resnet50(pretrained=True)

# Remove the last layer
# modules = list(resnet.children())[:-1]
# resnet = torch.nn.Sequential(*modules)

# Set model to evaluation mode
resnet.eval()

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define function to extract features from an image
def extract_features(image):
    # Convert grayscale images to color images
    if image.mode == 'L':
        image = Image.merge('RGB', [image] * 3)

    # Apply transformations
    image = transform(image).unsqueeze(0)

    # Pass the image through the model
    with torch.no_grad():
        features = resnet(image)

    return features


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# for i,file in enumerate(test_data.filename):
#     image = Image.open("sentiment_images/"+file)

#     if i==0:
#         with torch.no_grad():
#             image_features=extract_features(image).reshape(1000)
#     else :
#         with torch.no_grad():
#             image_features=torch.concat([image_features,extract_features(image).reshape(1000)])
#     if i%100==0:
#         print(i)
# torch.save(image_features,"testimgfeatures.pt")
test_image_features=torch.load("/content/drive/MyDrive/AIProject/testimgfeatures.pt")

In [8]:
# for i,file in enumerate(train_data.filename):
#     image = Image.open("sentiment_images/"+file)

#     if i==0:
#         with torch.no_grad():
#             image_features=extract_features(image).reshape(1000)
#     else :
#         with torch.no_grad():
#             image_features=torch.concat([image_features,extract_features(image).reshape(1000)])
#     if i%100==0:
#         print(i)
# torch.save(image_features,"trainimgfeatures.pt")
train_image_features=torch.load("/content/drive/MyDrive/AIProject/trainimgfeatures.pt")

In [9]:
# from transformers import BertTokenizer, BertModel


# # Load pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# # Load text dataset

# # Initialize tensor of zeros with appropriate shape
# features = torch.zeros((len(test_data), 768))

# # Loop over texts and extract features
# for i, text in enumerate(test_data.raw):
#     # Tokenize text
#     tokens = tokenizer.encode(text, add_special_tokens=True)

#     # Convert tokens to tensor
#     input_ids = torch.tensor([tokens])

#     # Pass input through BERT model
#     with torch.no_grad():
#         outputs = model(input_ids)

#     # Extract final hidden state of the first token (CLS token)
#     features[i] = outputs.last_hidden_state[0][0]
#     if i % 100==0:
#       print(i)

# # Print shape of features tensor
# print(features.shape) # Output: torch.Size([15912, 768])
# torch.save(features,"testtextfeatures.pt")
test_text_features=torch.load("/content/drive/MyDrive/AIProject/testtextfeatures.pt")


In [10]:
# features = torch.zeros((len(train_data), 768))
# for i, text in enumerate(train_data.raw):
#     # Tokenize text
#     tokens = tokenizer.encode(text, add_special_tokens=True)

#     # Convert tokens to tensor
#     input_ids = torch.tensor([tokens])

#     # Pass input through BERT model
#     with torch.no_grad():
#         outputs = model(input_ids)

#     # Extract final hidden state of the first token (CLS token)
#     features[i] = outputs.last_hidden_state[0][0]
#     if i % 100==0:
#       print(i)

# # Print shape of features tensor
# print(features.shape) # Output: torch.Size([15912, 768])
# torch.save(features,"traintextfeatures.pt")
train_text_features=torch.load("/content/drive/MyDrive/AIProject/traintextfeatures.pt")

# **Defining our MultiModel 1 which uses sentence level features of text and Image features**

In [11]:
class Multimodal(nn.Module):
    def __init__(self):
        super().__init__()
        self.textMod = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        self.imageMod = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        self.lastMod = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        self.ReLU = nn.ReLU()

    def forward(self, text, images):
        m1 = self.textMod(text)
        m2 = self.imageMod(images)
        out = torch.cat([m1, m2], dim=1)
        out = self.lastMod(out)
        return out


In [12]:
trainlabels=torch.tensor(train_data.sentiment,dtype=torch.float32)
trainlabels= trainlabels.reshape(len(train_data),1)

testlabels=torch.tensor(list(test_data.sentiment),dtype=torch.float32)
testlabels= testlabels.reshape(len(test_data),1)

In [13]:
test_image_features.shape

torch.Size([15912000])

In [14]:
train_image_features= train_image_features.reshape(19307,1000)
test_image_features= test_image_features.reshape(15912,1000)

In [15]:
train_loader=data_utils.DataLoader(data_utils.TensorDataset(train_image_features,train_text_features,trainlabels),batch_size=32)
test_loader=data_utils.DataLoader(data_utils.TensorDataset(test_image_features,test_text_features,testlabels),batch_size=32)

In [16]:
train_image_features.shape, train_text_features.shape,trainlabels.shape

(torch.Size([19307, 1000]), torch.Size([19307, 768]), torch.Size([19307, 1]))

In [17]:
len(train_data)

19307

In [23]:
mod= Multimodal()
device = torch.device('cpu')
mod.to(device)


Multimodal(
  (textMod): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
  )
  (imageMod): Sequential(
    (0): Linear(in_features=1000, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
  )
  (lastMod): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
  (ReLU): ReLU()
)

In [24]:
loss_func = nn.BCELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(mod.parameters(), lr=learning_rate)
epochs = 90

**Training**

In [ ]:
import numpy as np
mod.train()
mod.train()
for epoch in range(epochs):
    epoch_loss = []
    for x1, x2, l in train_loader:
        x1, x2, l = x1.to(device), x2.to(device), l.to(device)
        optimizer.zero_grad()
        y_pred = mod(x2, x1)
        loss = loss_func(y_pred, l)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())

    with torch.no_grad():
        A = mod(train_text_features, train_image_features) >= 0.5
        b = trainlabels >= 0.5
        train_ac = torch.sum(b == A).item() / len(train_image_features)

        A = mod(test_text_features, test_image_features) >= 0.5
        b = testlabels >= 0.5
        test_ac = torch.sum(b == A).item() / len(test_image_features)

    if epoch % 5 == 0:
        print('Epoch', epoch)
        print(f'Train Loss: {np.mean(epoch_loss)}  Test Accuracy: {float(test_ac)}  Train Accuracy: {float(train_ac)}')
        # print('classification report:\n', classification_report(np.array(b.cpu()), np.array(A.cpu())))

print('Final loss value: ' + str(loss.item()))

Epoch 0
Train Loss: 0.6373075234475515  Test Accuracy: 0.5480769230769231  Train Accuracy: 0.5395970373439685


# **Model 2 for word level textual and Image features multimodal**

**Preprocssing.** Here we are using CountVectorizer with ngrams as our word level textual feature extractor and using the same Image features from Resnet-50

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/AIProject/sentiment.csv')

# Set up the ngram vectorizer
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3),max_features=768)

# Fit the vectorizer on the dataset to build the vocabulary
ngram_vectorizer.fit(df.raw)

train_captions= list(train_data.raw)
test_captions= list(test_data.raw)

# Transform the dataset to create the feature vectors
train_feature_vectors = torch.tensor(ngram_vectorizer.transform(train_captions).toarray())
test_feature_vectors = torch.tensor(ngram_vectorizer.transform(test_captions).toarray())

# Print the shape of the feature vectors matrix
print(train_feature_vectors.shape)
print(test_feature_vectors.shape)



In [ ]:
train_feature_vectors = train_feature_vectors.to(torch.float32)
test_feature_vectors = test_feature_vectors.to(torch.float32)

In [ ]:
train_loader=data_utils.DataLoader(data_utils.TensorDataset(train_image_features,train_feature_vectors,trainlabels),batch_size=32)
test_loader=data_utils.DataLoader(data_utils.TensorDataset(test_image_features,test_feature_vectors,testlabels),batch_size=32)

**Defining our multimodal model**

In [ ]:
class Multimodal2(nn.Module):
    def __init__(self):
        super().__init__()
        self.textMod = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        self.imageMod = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
        )

        self.lastMod = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

        self.ReLU = nn.ReLU()

    def forward(self, text, images):
        m1 = self.textMod(text)
        m2 = self.imageMod(images)
        out = torch.cat([m1, m2], dim=1)
        out = self.lastMod(out)
        return out


In [ ]:
mod= Multimodal2()
device = torch.device('cpu')
mod.to(device)
loss_func = nn.BCELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(mod.parameters(), lr=learning_rate)
epochs = 1


**Training**

In [25]:
mod.train()
for epoch in range(90):
    for x1,x2,l in train_loader:
        x1,x2,l=x1.to(device),x2.to(device),l.to(device)
        optimizer.zero_grad()
        y_pred = mod(x2,x1)
        loss = loss_func(y_pred, l)
        loss.backward()
        optimizer.step()

    A=mod(train_text_features,train_image_features)>=0.5
    b=trainlabels>=0.5
    train_ac=sum(b==A)/len(train_image_features)
    A=mod(test_text_features,test_image_features)>=0.5
    b=testlabels>=0.5
    test_ac=sum(b==A)/len(test_image_features)
    if epoch % 5==0:
      print('Epoch',(epoch))
      print(f'Train Loss: {np.mean(epoch_loss)}  Test Accuracy: {float(test_ac)}  Train Accuracy: {float(train_ac)}')
        #print('clasification report:\n', classification_report(np.array(b),np.array(A)))
print('Final loss value: '+str(loss.item()))

NameError: name 'train_loader' is not defined